In [1]:
using JSON;
using DataFrames;
using CSV;

# Read JSON file, create Dataframe and learn to access it

In [2]:
# read the JSON file line by line 
p = "coronavirus-tweet-id-2020-01-22-06-11.json"
@time jsons = open(p) do io
    JSON.parse.(eachline(io))
end;

  0.384163 seconds (1.50 M allocations: 121.597 MiB, 27.12% gc time)


In [3]:
# Create a list of keys 
tweet_keys = []
for key in keys(jsons[1])
    push!(tweet_keys, key)
end

In [4]:
# a clean tweet object extracts and contains the following attributes from the raw tweet: 
# the original tweet's : id, created at, user, language text

# if the tweet is a retweet, our object also contains the retweet's: rt_text, rt_user, rt_id, rt_lang
# along with a boolean value is_rt that indicates whether the current tweet is a retweet. 

function clean_tweet(jsons)
    IDs = []
    created_at = []
    user_names = []
    hashtags = []


    retweeted_status = [] 
    rt_user = [] 
    rt_id = []

    for i in 1:length(jsons)
        push!(IDs, jsons[i]["id"])
        push!(created_at, jsons[i]["created_at"])
        push!(user_names, jsons[i]["user"]["screen_name"])
        
        #check if retweeted 
        if haskey(jsons[i], "retweeted_status")
            push!(retweeted_status, true)
            
            # if there are hashtags, push them 
            if length(jsons[i]["retweeted_status"]["entities"]["hashtags"]) != 0
                # read all hashtags and safe in temp_hashs
                temp_hashs = []
                for j in 1:length(jsons[i]["retweeted_status"]["entities"]["hashtags"])
                    push!(temp_hashs,jsons[i]["retweeted_status"]["entities"]["hashtags"][j]["text"])
                end
                # push hashtags into hashtag array 
                push!(hashtags,temp_hashs)
            else 
                push!(hashtags,0)
            end
                    
            # original Tweeter 
            push!(rt_user, jsons[i]["retweeted_status"]["user"]["screen_name"])
            push!(rt_id, jsons[i]["retweeted_status"]["id"])
            
        # if it wasn't a retweet
        else
            
            # if there are hashtags, push them 
            if length(jsons[i]["entities"]["hashtags"]) != 0
                temp_hashs = []
                for j in 1:length(jsons[i]["entities"]["hashtags"])
                    push!(temp_hashs,jsons[i]["entities"]["hashtags"][j]["text"])
                end
                push!(hashtags,temp_hashs)
            else 
                push!(hashtags,0)
            end
            
            push!(retweeted_status, false)
            push!(rt_user, 0)
            push!(rt_id, 0)
        end
    end
    
    return IDs , created_at, user_names, retweeted_status, rt_user,  rt_id, hashtags
end

clean_tweet (generic function with 1 method)

In [5]:
#Filter relevant information via clean_tweet() function
@time IDs , created_at, user_names, retweeted_status, rt_user,  rt_id, hashtags = clean_tweet(jsons);

  0.118989 seconds (81.44 k allocations: 4.125 MiB)


In [6]:
# create a DataFrame with all wanted information
df = DataFrame(ID = IDs[1:length(IDs)], Created_at = created_at[1:length(created_at)],USER = user_names[1:length(user_names)], Is_Retweet = retweeted_status[1:length(retweeted_status)], RT_User = rt_user[1:length(rt_user)], RT_Id = rt_id[1:length(rt_id)], Hashtags = hashtags[1:length(hashtags)]);

In [7]:
# Examples for how to access the df (time is not ordered yet -> see below)
println("Time of first tweet: ",df.Created_at[1])

Time of first tweet: Wed Jan 22 06:27:54 +0000 2020


In [8]:
# Number of Tweets in time period
println("Tweets in total: ", length(df.Hashtags))

Tweets in total: 2847


In [9]:
# Write df into CSV
CSV.write("ExampleCSV.csv",df)

"ExampleCSV.csv"

In [10]:
# Read CSV (3 columns are not shown) directly into dataframe
example_df = DataFrame(CSV.File("ExampleCSV.csv"))

,ID,Created_at,USER,Is_Retweet
,Int64,String,String,Bool
1,1219869278574387200,Wed Jan 22 06:27:54 +0000 2020,UsaCleanup,true
2,1219871372182245400,Wed Jan 22 06:36:13 +0000 2020,innovationville,false
3,1219875910641488000,Wed Jan 22 06:54:15 +0000 2020,HelsinkiOne,false
4,1219867508041105400,Wed Jan 22 06:20:52 +0000 2020,ivona_11,true
5,1219868124633190400,Wed Jan 22 06:23:19 +0000 2020,sarahderpy,true
6,1219872204571259000,Wed Jan 22 06:39:32 +0000 2020,Frankdiplomat,false
7,1219871769269547000,Wed Jan 22 06:37:48 +0000 2020,evaArtemise97,false
8,1219871144649556000,Wed Jan 22 06:35:19 +0000 2020,Johanisms,true
9,1219872618787946500,Wed Jan 22 06:41:11 +0000 2020,RedMystique,true


In [11]:
# Show first 5 rows with all columns 
show(first(example_df,5), allcols=true)

5×7 DataFrame
│ Row │ ID                  │ Created_at                     │ USER            │
│     │ Int64               │ String                         │ String          │
├─────┼─────────────────────┼────────────────────────────────┼─────────────────┤
│ 1   │ 1219869278574387200 │ Wed Jan 22 06:27:54 +0000 2020 │ UsaCleanup      │
│ 2   │ 1219871372182245400 │ Wed Jan 22 06:36:13 +0000 2020 │ innovationville │
│ 3   │ 1219875910641488000 │ Wed Jan 22 06:54:15 +0000 2020 │ HelsinkiOne     │
│ 4   │ 1219867508041105400 │ Wed Jan 22 06:20:52 +0000 2020 │ ivona_11        │
│ 5   │ 1219868124633190400 │ Wed Jan 22 06:23:19 +0000 2020 │ sarahderpy      │

│ Row │ Is_Retweet │ RT_User │ RT_Id               │
│     │ Bool       │ String  │ Int64               │
├─────┼────────────┼─────────┼─────────────────────┤
│ 1   │ true       │ Reuters │ 1219854705330999300 │
│ 2   │ false      │ 0       │ 0                   │
│ 3   │ false      │ 0       │ 0                   │
│ 4   │ true       

In [12]:
# Check format of the Hashtag Arrays 
typeof(example_df.Hashtags[9])

String

In [13]:
# Parse Hashtag entry from String to Array  
eval(Meta.parse(example_df.Hashtags[9])) 

3-element Array{Any,1}:
 "sb276"       
 "vaccinetruth"
 "vaccines"    

In [14]:
# Parse whole Hashtag column back to array 
example_df.Hashtags = eval.(Meta.parse.(example_df.Hashtags[1:end]));

In [15]:
# Check cleaned column
example_df.Hashtags[9]

3-element Array{Any,1}:
 "sb276"       
 "vaccinetruth"
 "vaccines"    

# Format the "Created_at" column and sort it 

In [16]:
# Useful package 
using Dates

In [17]:
# Check previous date format
example_df.Created_at[1]

"Wed Jan 22 06:27:54 +0000 2020"

In [18]:
# Format the dates to read it easily with the Dates.pkg and safe it in the dataframe
example_df.Created_at = Dates.DateTime.(example_df.Created_at, "e u dd HH:MM:SS +0000 yyyy");

In [19]:
# Sort array by date column 
df_sorted = sort(df, order(:Created_at));
# Print first and last date 
println(df_sorted.Created_at[1])
println(df_sorted.Created_at[end])

Wed Jan 22 06:00:00 +0000 2020
Wed Jan 22 11:59:56 +0000 2020
